# Calculations for FOV

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mgrandau/astrophotography-camera-exposure-times/blob/main/astrophotography-camera-exposure-times%20.ipynb)


This notebook goes into the calculations for the "Field of View" of my cameras. This should allow me to calculate a box that can be put over star maps to see the expected view.

$\textbf{FOV\ (arcseconds)} = \dfrac{Chip\ Size}{Focal\ Length} * \dfrac{360^{\circ}}{2\pi} * 60\ \dfrac{arcminutes}{1^{\circ}} * 60\ \dfrac{arcseconds}{arcminute}$

In [5]:
import math

# My telescope has 1600mm focal length
focal_length = 1600

In [6]:
def fov(size_millimeter) :   
    return (size_millimeter/focal_length) * (360/(2*math.pi)) * 60 * 60

## ASI482MC

So for the ASI482MC:
* Resolution: 2.07Mega Pixel,1920 x 1080
* Pixel Size: 5.8um x 5.8um
* Dimension of sensor: 11.13mm x 6.26mm

In [7]:
ASI482MC_size_millimeter = (11.13, 6.26)
ASI482MC_pixel_size_millimeter = 5.8/1000
ASI482MC_FOV_pixel_in_arcseconds = fov(ASI482MC_pixel_size_millimeter)

In [8]:
print(f'(ASI482MC) The theoretical FOV is: {fov(ASI482MC_size_millimeter[0])/60:.2f} x {fov(ASI482MC_size_millimeter[1])/60:.2f} arcmin')

(ASI482MC) The theoretical FOV is: 23.91 x 13.45 arcmin


We will want to know the field of view per pixel for later.

In [9]:
print(f'(ASI482MC) The theoretical FOV per pixel is: {ASI482MC_FOV_pixel_in_arcseconds:.3f} arcseconds')

(ASI482MC) The theoretical FOV per pixel is: 0.748 arcseconds


## ASI120MC-S

So for the ASI120MC-S:
* Resolution: 1.2Mega Pixels 1280×960
* Pixel Size: 3.75um x 3.75um
* Dimension of sensor: 4.8mm x 3.6mm

In [10]:
ASI120MC_S_size_millimeter = (4.8, 3.6)
ASI120MC_S_pixel_size_millimeter = 4.8/1000
ASI120MC_S_FOV_pixel_in_arcseconds = fov(ASI120MC_S_pixel_size_millimeter)

In [11]:
print(f'(ASI120MC-S) The theoretical FOV is: {fov(ASI120MC_S_size_millimeter[0])/60:.2f} x {fov(ASI120MC_S_size_millimeter[1])/60:.2f} arcmin')

(ASI120MC-S) The theoretical FOV is: 10.31 x 7.73 arcmin


We will want to know the field of view per pixel for later.

In [12]:
print(f'(ASI120MC-S) The theoretical FOV per pixel is: {ASI120MC_S_FOV_pixel_in_arcseconds:.3f} arcseconds')

(ASI120MC-S) The theoretical FOV per pixel is: 0.619 arcseconds


## 150 degree "All Sky Lens" attachment

The ASI120MC-S has an "all sky" lens attachment.  

$\textbf{FOV\ (arcseconds)} = \textbf{FOV\ lens\ (degrees)} * 60\ \dfrac{arcminutes}{1^{\circ}} * 60\ \dfrac{arcseconds}{arcminute}$


In [38]:
all_sky_FOV_in_arcseconds = 150 * 60 * 60
all_sky_FOV_pixel_in_arcseconds = all_sky_FOV_in_arcseconds / 1280

An estimate of the FOV of the pixel can be calculated from the resolution of the camera. It should give a close approximation.

In [39]:
print(f'(ASI120MC-S All Sky) The theoretical FOV per pixel is: {all_sky_FOV_pixel_in_arcseconds:.0f} arcseconds')

(ASI120MC-S All Sky) The theoretical FOV per pixel is: 422 arcseconds


## Rotation of earth (arcseconds per second) 

$\textbf{Single\ Rotation\ (arcseconds)} = \dfrac{360^{\circ}}{2\pi} * 60\ \dfrac{arcminutes}{1^{\circ}} * 60\ \dfrac{arcseconds}{arcminute}$

$\textbf{Single\ Rotation\ (seconds)} = 24\ hr * 60\ \dfrac{min}{hr} * 60\ \dfrac{seconds}{min}$ 

The actual rotation in seconds is based on sidereal rate (23 hr 56 min 4s). It's the whole reason we have leap years. Because of this the sidereal time can be calculated: 

$\textbf{Single\ Rotation\ (sidereal\ seconds)} = (((23\ hr * 60\ \dfrac{min}{hr}) + 56\ min) * 60\ \dfrac{seconds}{min}) + 4\ seconds$

$\textbf{Rotation\ of\ earth\ (arcseconds\ per\ second)} = \dfrac{Single\ Rotation\ (arcseconds)}{Single\ Rotation\ (sidereal\ seconds)}$

In [15]:
rotation_arcseconds = (360/(2*math.pi)) * 60 * 60
rotation_arcseconds

206264.80624709633

In [16]:
rotation_seconds = 24 * 60 * 60
rotation_seconds

86400

In [17]:
sidereal_rotation_seconds = (((23 * 60)+56) * 60) +4
sidereal_rotation_seconds

86164

In [18]:
rotation_rate = rotation_arcseconds/sidereal_rotation_seconds
print(f'The rotational rate of earth is: {rotation_rate:.3f} arcseconds/second')

The rotational rate of earth is: 2.394 arcseconds/second


So a star on the celestial equator will have a length of 2.4 arcseconds and a star above the axis of rotation will not move.

## Calculating max exposure time

The idea is to calculate the maximum exposure time based on the star's declination (The distance in degrees from the celestial equator). 

The maximum exposure time is the amount of time to hold a point of light in a particular pixel on the CCD.

We do this to optimize the color absorption. The longer we can expose a pixel to the light the less atmospheric noise will matter. These effects could be light clouds, thermal effects or other phenomena. The goal is to average out a better signal to noise.


### Why does a stars declination matter?

The Polaris (north star) declination is 89° 15′ 50.8″ while Regulus declination is 11° 58′ 01.95″. The distance a star moves over time relative to a pixel on the camera is based on it's declination from the celestial equator. Polaris barely moves while Regulus while will appear to move farther or hav a greater arc across the sky.

Or in other words Polaris can have a longer exposure time than Regulus.

In other words:


$\textbf{declination\ (degrees)} = ((((\textbf{value\ (degrees)} * 60\ \dfrac{arcminutes}{1^{\circ}}) + \textbf{value\ (arcminutes)}) * 60\ \dfrac{arcseconds}{arcminute})+ \textbf{value\ (arcseconds)}) * \dfrac{1^{\circ}}{3600\ (arcseconds)}$

$\textbf{stars\ expected\ movement\ (arcseconds per second)} = rotational\ rate * \dfrac{90^{\circ} - \lvert \textbf{declination\ (degrees)} \rvert}{90^{\circ}}$

In [19]:
# Polaris (north star) is at 89° 15′ 50.8″ 
degrees = 89
minutes = 15
seconds = 50.8

DEC = ((((degrees*60)+minutes)*60)+seconds)/3600
DEC

89.2641111111111

In [20]:
polaris_movement_per_second = rotation_rate*((90-abs(DEC))/90)
polaris_movement_per_second

0.01957352375651326

In [21]:
# Regulus (brightest in Leo) is at 11° 58′ 1.95″ 
degrees = 11
minutes = 58
seconds = 1.95

DEC = ((((degrees*60)+minutes)*60)+seconds)/3600
DEC

11.967208333333332

In [22]:
regulus_movement_per_second = rotation_rate*((90-abs(DEC))/90)
regulus_movement_per_second

2.0755534219040994

In [23]:
# Celestial Equator is at 0° 0′ 0″ 
degrees = 0
minutes = 0
seconds = 1

DEC = ((((degrees*60)+minutes)*60)+seconds)/3600
DEC

0.0002777777777777778

In [24]:
near_celestial_equator_movement_per_second  = rotation_rate*((90-abs(DEC))/90)
near_celestial_equator_movement_per_second 

2.393855550198737

As you can see Regulus will move 2 arcseconds while Polaris will barely move across the sky. 

### So what is the max exposure time?

So depending on a camera's field of view per pixel, the expected movement of a star in seconds can be used to calculate the theoretical max exposure time.

$\textbf{max\ exposure\ time\ (second)} = \dfrac{FOV\ per\ pixel\ (arcseconds)}{stars\ movement\ (arcseconds\ per\ second)}$ 

In [25]:
# Max exposure for ASI482MC on Polaris in seconds
max_exposure = ASI482MC_FOV_pixel_in_arcseconds/polaris_movement_per_second
print(f'(ASI482MC) Polaris exposure (seconds): {max_exposure:.0f}')

(ASI482MC) Polaris exposure (seconds): 38


In [26]:
# Max exposure for ASI482MC on Regulus in milliseconds
max_exposure = (ASI482MC_FOV_pixel_in_arcseconds/regulus_movement_per_second) * 1000
print(f'(ASI482MC) Regulus exposure (milliseconds): {max_exposure:.0f}')

(ASI482MC) Regulus exposure (milliseconds): 360


In [42]:
# Max exposure for ASI482MC at celestial equator in milliseconds
max_exposure = (ASI482MC_FOV_pixel_in_arcseconds/rotation_rate) * 1000
print(f'(ASI482MC) Celestial equator exposure (milliseconds): {max_exposure:.0f}')

(ASI482MC) Celestial equator exposure (milliseconds): 312


In [27]:
# Max exposure for ASI120MC-S on Polaris in seconds
max_exposure = ASI120MC_S_FOV_pixel_in_arcseconds/polaris_movement_per_second
print(f'(ASI120MC-S) Polaris exposure (seconds): {max_exposure:.0f}')

(ASI120MC-S) Polaris exposure (seconds): 32


In [43]:
# Max exposure for ASI120MC-S on Regulus in milliseconds
max_exposure = (ASI120MC_S_FOV_pixel_in_arcseconds/regulus_movement_per_second) * 1000
print(f'(ASI120MC-S) Regulus exposure (milliseconds): {max_exposure:.0f}')


(ASI120MC-S) Regulus exposure (milliseconds): 298


We can also calculate the best exposure time for the "all sky" attachment to the ASI120MC-S. Because the image will contain some star on the celestial equator, we set the exposure based on those stars (rotational rate a 0 Declination). 

In [40]:
max_exposure = all_sky_FOV_pixel_in_arcseconds/rotation_rate
print(f'(ASI120MC-S) All sky exposure (seconds): {max_exposure:.0f}')

(ASI120MC-S) All sky exposure (seconds): 176


# Warning: These are theoretical limits are meant to get you near max exposure time. Being a little less should help maximize results.